<a href="https://colab.research.google.com/github/thineikhaing/BigDataSparkPG/blob/main/SparkSQLPlayGround.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Installation**
The first step involves installing pyspark. The next step is to install findspark library.

In [5]:
# install pyspark using pip
!pip install --ignore-install -q pyspark
# install findspark using pip
!pip install --ignore-install -q findspark

## Mounting Google Drive

You can use the drive module from google.colab to mount your entire Google Drive to Colab by executing the below code. This will provide us with an authentication link to connect to Google Drive. Choose the Google account whose Drive you want to mount. Allow Google Drive Stream access to your Google Account.

In [14]:
# to read in data from a text file, first upload the data file into your google drive and then mount your google drive onto colab
from google.colab import drive
# to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True)
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


Run the sample spark df

**Day3 Workshop4**

Exercise1:

Using Dataframes and SparkSQL and working on Rebu Case study files mentioned above, write Spark SQL
for the following:

A. Data retrieval using Spark SQL
1. Retrieve all Driver data (use Drivers.CSV)
2. Retrieve all Taxis and display the data in ascending order of Taxi License Plate number.
3. Retrieve all Limosine Taxies. You should display only the Taxi Number, Taxi Type, and Taxi
Colour.
4. Retrieve all 4 seater Premier taxis.

B. Aggregation and Statistical Queries (use BEAD_Rebu_TripData.CSV)
Determine the average distance per trip based on ALL trips in the month of January 2024.
Find the total fares collected grouped by Taxi Type Maxi Cab

C. Analytics Questions
7. Determine the Average Occupancy i.e., (Number of Passengers / Passenger Capacity) for
Standard Taxis.
8. Determine Fares Collected by Day of the Week (ie., Sun, Mon, Tue) for the month of Jan 2024.
9. Prepare a Tabulation report showing total revenue against the two dimensions Hour of the
day AND Day of the Week.
10. Compare the total number of trips per day made by all taxis in weekends vs the total number
of trips made per day during weekdays in the month of Jan 2024.

D. Multiple Entities Joining and multiple formats joined in a DataFrame
11. Determine the total fares paid by all Gold Status Passengers in the month of Jan 2024. What
percentage does this make from the total fares for all customers in month of Jan 2024.

In [107]:
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import avg, sum, col, month, year, to_date, dayofweek

spark = SparkSession.builder.appName("RebuCaseStudy").getOrCreate()

drivers_df = spark.read.option("header", "true").csv("/content/drive/My Drive/BEAD_DATA/BEAD_Rebu_Drivers.csv")
taxis_df = spark.read.option("header", "true") \
                      .option("inferSchema", "true")\
                      .option("multiline", "true").json("/content/drive/My Drive/BEAD_DATA/BEAD_Rebu_TaxiCabs.json")
trip_data_df = spark.read.option("header", "true").csv("/content/drive/MyDrive/BEAD_DATA/BEAD_Rebu_TripData.csv")
passenger_df = spark.read.option("header", "true").csv("/content/drive/MyDrive/BEAD_DATA/BEAD_Rebu_Passengers.csv")

# Add a column Convert Trip Start Time to date format
trip_data_df = trip_data_df.withColumn("TripDate", to_date(trip_data_df["Date"], 'd-MMM-yy'))

# Add a column indicating if the day is a weekend or weekday
trip_data_df = trip_data_df.withColumn("IsWeekend", (dayofweek(trip_data_df.TripDate) == 1) | (dayofweek(trip_data_df.TripDate) == 7))

# Filter for trips in January 2024 and calculate the average distance traveled
jan_2024_trips = trip_data_df.filter((month(trip_data_df.TripDate) == 1) & (year(trip_data_df.TripDate) == 2024))


In [ ]:
# A - Data retrieval using Spark SQL
#===================================

# 1. Retrieve all Driver data
drivers_df.show()
# 2. Retrieve all Taxis and display the data in ascending order of Taxi License Plate number
taxis_df = taxis_df.orderBy("TaxiNumber")
taxis_df.show(taxis_df.count(),False)

# 3. Retrieve all Limousine Taxis (Taxi Number, Taxi Type, Taxi Colour)
limosine_taxis_df = taxis_df.select("TaxiNumber", "TaxiType", "TaxiColor").where("TaxiType = 'Limosine'")
limosine_taxis_df.show(limosine_taxis_df.count(),False)

# 4. Retrieve all 4-seater Premier taxis
four_seater_taxi = taxis_df.where("TaxiType = 'Premier' AND TaxiPassengerCapacity = 4")
four_seater_taxi.show(four_seater_taxi.count(),False)

print(f"Total count of records in trip_data_df: {taxis_df.count()}")
print(f"Total count of records in limosine_taxis_df: {limosine_taxis_df.count()}")
print(f"Total count of records in four_seater_taxi: {four_seater_taxi.count()}")

In [ ]:
# B - Aggregation and Statistical Queries
#========================================
# 5. Determine the average distance per trip based on ALL trips in the month of January 2024
average_distance_jan = jan_2024_trips.agg(avg("Distance Travelled").alias("AverageDistance"))
average_distance_jan.show()

# 6 Filter for Maxi Cab trips and calculate the total fares collected
total_fares_maxi_cab = trip_data_df.filter(trip_data_df["Taxi Type"] == 'Maxi Cab') \
                                   .groupBy("Taxi Type") \
                                   .agg(sum("Trip Fare").alias("TotalFare"))
total_fares_maxi_cab.show()

In [ ]:
# C. Analytics
#=============
# 7 Filter for Standard Taxis and calculate the average occupancy
average_occupancy = trip_data_df.filter(trip_data_df["Taxi Type"] == 'Standard') \
                                .withColumn("Occupancy", col("Number Of Passengers") / col("Taxi Capacity")) \
                                .agg(avg("Occupancy").alias("AverageOccupancy")).show()

# 8. Filter for trips in January 2024 and calculate total fares collected by day of the week
fares_by_day_of_week = jan_2024_trips.groupBy("Day") \
                                   .agg(sum("Trip Fare").alias("TotalFare")).show()


# 9 . Filter for trips in January 2024 and calculate total revenue by hour of the day and day of the week
revenue_by_hour_day = jan_2024_trips.groupBy("Hour of Day", "Day") \
                                  .agg(sum("Trip Fare").alias("TotalRevenue")) \
                                  .orderBy("Hour of Day", "Day")
revenue_by_hour_day.show()

# 10. Filter for trips in January 2024 and count trips per day, grouping by weekends and weekdays
trips_weekends_vs_weekdays = trip_data_df.groupBy("IsWeekend") \
                                         .agg((count("*") / countDistinct("Date")).alias("AverageTripsPerDay")).show()

print(f"Total count of records in Revenue by hour day: {revenue_by_hour_day.count()}")

In [ ]:
# D Multiple Entities Joining and multiple formats joined in a DataFrame
#========================================================================

# Rename the CustomerId column to Passenger ID to match the trip data
passenger_df = passenger_df.withColumnRenamed("PassengerID", "Passenger ID")

# joined_df = trip_data_df.join(customer_df, trip_data_df["Passenger ID"] == customer_df["Passenger ID"], "inner")
joined_df = trip_data_df.join(passenger_df, "Passenger ID")

# Filter for Gold Status Passengers in January 2024 and calculate total fares
total_fares_gold = joined_df.filter((col("MemSilvererStGoldtus") == "Gold") &
                                    (month(joined_df.TripDate) == 1) &
                                    (year(joined_df.TripDate) == 2024)) \
                            .agg(sum("Trip Fare").alias("TotalFaresGold"))

# Calculate total fares for all passengers in January 2024
total_fares_all = joined_df.filter((month(joined_df.TripDate) == 1) &
                                   (year(joined_df.TripDate) == 2024)) \
                           .agg(sum("Trip Fare").alias("TotalFaresAll"))

total_fares_gold_value = total_fares_gold.collect()[0]["TotalFaresGold"]
total_fares_all_value = total_fares_all.collect()[0]["TotalFaresAll"]

# Calculate the percentage
percentage_gold = (total_fares_gold_value / total_fares_all_value) * 100

print(f"Total fares paid by Gold Status Passengers in January 2024: {total_fares_gold_value}")
print(f"Percentage of total fares for all customers: {percentage_gold:.2f}%")
